<a href="https://colab.research.google.com/github/vanesalinas/HenryPI_MLOps_PT09/blob/main/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [39]:
import pandas as pd
import ast
import json

# **Una primera mirada al dataset**

---



In [3]:
ruta_movies = '/content/drive/MyDrive/Datasets_PI_MLOPS/movies_dataset.csv'
ruta_credits = '/content/drive/MyDrive/Datasets_PI_MLOPS/credits.csv'

In [4]:
df_movies = pd.read_csv(ruta_movies)
df_credits = pd.read_csv(ruta_credits)

<ipython-input-4-e536d47ac3da>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv(ruta_movies)


In [81]:
df_movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [6]:
df_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [103]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

# **Limpieza de datos**

---


## Datos anidados

Podemos ver que en ambos dataframe hay columnas con datos anidados. De todas ellas las que nos proporcionan infomacion relevante para nuestro analisis son 'genres' del df_movies y 'cast' y 'crew' del df_credits por lo que solo desanidaremos los datos que se encuentran en ellas.

In [83]:
df_movies['genres'][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

Creamos una copia de df_movies para no modificar el dataframe original y evaluamos la columna 'genres' con el metodo ast.literal_eval para poder trabajar con las listas de diccionarios. Inicializamos una lista vacia que ira almacenando los nombres de los generos que se encuentran dentro de las listas de la columna 'genres'. Luego la agregamos al dataframe y eliminamos la columna 'genre'.

In [140]:
df = df_movies.copy()

# Convertir la columna 'genres' a listas de diccionarios
df['genres'] = df['genres'].apply(ast.literal_eval)

# Inicializar una lista vacías para crear una nueva columna
genres_names = []

# Iterar sobre las listas de diccionarios y extraer los datos en nuevas columnas
for linea in df['genres']:
    nombres = [genres['name'] for genres in linea]

    genres_names.append(nombres)

# Añadir las nuevas columnas al DataFrame
df['genre_names'] = genres_names
df = df.drop('genres', axis=1)

df.head()


,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,genre_names
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[Animation, Comedy, Family]"
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[Adventure, Fantasy, Family]"
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[Romance, Comedy]"
3,False,NaN,16000000,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[Comedy, Drama, Romance]"
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,[Comedy]


Evaluamos una celda de la columna 'cast' y una de la columna 'crew'

In [37]:
df_credits['cast'][0]

[{'cast_id': 14,
  'character': 'Woody (voice)',
  'credit_id': '52fe4284c3a36847f8024f95',
  'gender': 2,
  'id': 31,
  'name': 'Tom Hanks',
  'order': 0,
  'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'},
 {'cast_id': 15,
  'character': 'Buzz Lightyear (voice)',
  'credit_id': '52fe4284c3a36847f8024f99',
  'gender': 2,
  'id': 12898,
  'name': 'Tim Allen',
  'order': 1,
  'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'},
 {'cast_id': 16,
  'character': 'Mr. Potato Head (voice)',
  'credit_id': '52fe4284c3a36847f8024f9d',
  'gender': 2,
  'id': 7167,
  'name': 'Don Rickles',
  'order': 2,
  'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'},
 {'cast_id': 17,
  'character': 'Slinky Dog (voice)',
  'credit_id': '52fe4284c3a36847f8024fa1',
  'gender': 2,
  'id': 12899,
  'name': 'Jim Varney',
  'order': 3,
  'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'},
 {'cast_id': 18,
  'character': 'Rex (voice)',
  'credit_id': '52fe4284c3a36847f8024fa5',
  'gender': 2,
  'id': 12900,
 

In [20]:
df_credits['crew'][0]

'[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name\': \'A

En ambas hay mucha informacion irrelevante, lo cual al expandir los datos nos generaria dataframes muy extensos; por lo que solo vamos a extraer el nombre de los actores, los directores y el respectivo id de pelicula de cada uno ya que esta es la informacion necesaria para nuestro analisis.

In [22]:
df_credits['crew'] = df_credits['crew'].apply(ast.literal_eval)

In [25]:
df_credits['cast'] = df_credits['cast'].apply(ast.literal_eval)

In [156]:
# Crear listas vacías para almacenar datos expandidos
directors = []
actors = []

# Iterar sobre cada película en el DataFrame
for index, row in df_credits.iterrows():
    movie_id = row['id']
    cast_list = row['cast']
    crew_list = row['crew']

    # Iterar sobre el elenco y el equipo de la película
    for member in cast_list:
        actor_data = {
          'id': movie_id,
          'actor_name': member['name'],
        }
        actors.append(actor_data)

    for member in crew_list:
        if member['job'] == 'Director':
            director_data = {
                'id': movie_id,
                'director_name': member['name'],
            }
            directors.append(director_data)

# Convertir las listas a DataFrames
df_directors = pd.DataFrame(directors)
df_actors = pd.DataFrame(actors)

In [157]:
df_directors.head()

,id,director_name
0,862,John Lasseter
1,8844,Joe Johnston
2,15602,Howard Deutch
3,31357,Forest Whitaker
4,11862,Charles Shyer


In [158]:
df_actors.head()

,id,actor_name
0,862,Tom Hanks
1,862,Tim Allen
2,862,Don Rickles
3,862,Jim Varney
4,862,Wallace Shawn


In [175]:
df_actors[df_actors['id'] == 862]

,id,actor_name
0,862,Tom Hanks
1,862,Tim Allen
2,862,Don Rickles
3,862,Jim Varney
4,862,Wallace Shawn
5,862,John Ratzenberger
6,862,Annie Potts
7,862,John Morris
8,862,Erik von Detten
9,862,Laurie Metcalf


## Columnas irrelevantes

Del dataframe de creditos se extrajo solo la informacion necesaria almacenandola en dos nuevos dataframe; pero del df_movies eliminaremos aquellas columnas que no aportan informacion para nuestra API ni para el analisis solicitado.

In [141]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'homepage', 'id', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'video', 'vote_average', 'vote_count',
       'genre_names'],
      dtype='object')

In [142]:
df.drop(['adult', 'belongs_to_collection', 'video', 'imdb_id', 'original_title', 'poster_path', 'homepage', 'original_language', 'runtime', 'spoken_languages', 'production_companies', 'production_countries'], axis=1, inplace=True)

## Campos vacios o nulos

In [143]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   budget        45466 non-null  object 
 1   id            45466 non-null  object 
 2   overview      44512 non-null  object 
 3   popularity    45461 non-null  object 
 4   release_date  45379 non-null  object 
 5   revenue       45460 non-null  float64
 6   status        45379 non-null  object 
 7   tagline       20412 non-null  object 
 8   title         45460 non-null  object 
 9   vote_average  45460 non-null  float64
 10  vote_count    45460 non-null  float64
 11  genre_names   45466 non-null  object 
dtypes: float64(3), object(9)
memory usage: 4.2+ MB


Los valores nulos de los campos revenue, budget seran rellenados por el número 0. Y los valores nulos del campo release date seran eliminados.

In [144]:
# Rellenar los valores nulos en las columnas 'revenue' y 'budget'
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)

# Eliminar filas donde 'release_date' es nulo
df.dropna(subset=['release_date'], inplace=True)

## Formato fecha

Las columnas con fechas tendran el formato AAAA-mm-dd. El unico campo con este tipo de datos es 'release_date'.
Ademas crearemos la columna 'release_year' donde se extraera el año de la fecha de estreno.

In [145]:
#verificamos el tipo de dato de la columna
df['release_date'].dtype

dtype('O')

In [146]:
df['release_date'][10394]

'2005-07-14'

In [152]:
# Convertir la columna 'release_date' a datetime
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')

# Crear la columna 'release_year'
df['release_year'] = df['release_date'].dt.year.fillna(0).astype(int)

df.head()

,budget,id,overview,popularity,release_date,revenue,status,tagline,title,vote_average,vote_count,genre_names,release_year
0,30000000,862,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,Released,NaN,Toy Story,7.7,5415.0,"[Animation, Comedy, Family]",1995
1,65000000,8844,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[Adventure, Fantasy, Family]",1995
2,0,15602,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,"[Romance, Comedy]",1995
3,16000000,31357,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,"[Comedy, Drama, Romance]",1995
4,0,11862,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,[Comedy],1995


In [153]:
df['release_date'].dtype

dtype('<M8[ns]')

## Return

Se crea la columna con el retorno de inversión, llamada return dividiendo 'revenue' por 'budget'; cuando no hay datos disponibles para calcularlo, tomara el valor 0.

Primero verificamos el tipo de datos de cada columna para saber si son numericos y asi poder efectuar operaciones aritmeticas.

In [166]:
df['revenue'].dtype

dtype('float64')

In [167]:
df['budget'].dtype

dtype('O')

In [168]:
# Convertir la columna 'budget' a números
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')

# Calcular el retorno de inversión
df['return'] = df.apply(lambda row: round(row['revenue'] / row['budget'],2) if row['budget'] != 0 else 0, axis=1)

df.head()

,budget,id,overview,popularity,release_date,revenue,status,tagline,title,vote_average,vote_count,genre_names,release_year,return
0,30000000.0,862.0,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,Released,NaN,Toy Story,7.7,5415.0,"[Animation, Comedy, Family]",1995,12.45
1,65000000.0,8844.0,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[Adventure, Fantasy, Family]",1995,4.04
2,0.0,15602.0,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,"[Romance, Comedy]",1995,0.00
3,16000000.0,31357.0,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,"[Comedy, Drama, Romance]",1995,5.09
4,0.0,11862.0,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,[Comedy],1995,0.00


# **Disponibilizacion de datos**

---

Se crearan dataframes mas pequeños con el fin de optimizar el funcionamiento de las consultas de la API que se diseñara con el fin de disponibilizar los datos.  

In [169]:
df.columns

Index(['budget', 'id', 'overview', 'popularity', 'release_date', 'revenue',
       'status', 'tagline', 'title', 'vote_average', 'vote_count',
       'genre_names', 'release_year', 'return'],
      dtype='object')

In [154]:
fecha_estreno = df[['release_date']]

In [155]:
score_votos = df[['title', 'release_year', 'popularity', 'vote_average', 'vote_count']]

In [160]:
df['id'].dtype

dtype('O')

In [161]:
df_actors['id'].dtype

dtype('int64')

In [170]:
# Agrupar actores por ID y crear una lista
df_actores_agrupado = df_actors.groupby('id')['actor_name'].agg(list).reset_index()

# Convertir la columna 'id' en df a numérico
df['id'] = pd.to_numeric(df['id'], errors='coerce')

# Concatenar los df en base a la columna 'id'
actores = pd.merge(df[['id', 'return']], df_actores_agrupado, on='id')

actores.head()

,id,return,actor_name
0,862.0,12.45,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,8844.0,4.04,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,15602.0,0.00,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop..."
3,31357.0,5.09,"[Whitney Houston, Angela Bassett, Loretta Devi..."
4,11862.0,0.00,"[Steve Martin, Diane Keaton, Martin Short, Kim..."


In [171]:
actores['actor_name'][0]

['Tom Hanks',
 'Tim Allen',
 'Don Rickles',
 'Jim Varney',
 'Wallace Shawn',
 'John Ratzenberger',
 'Annie Potts',
 'John Morris',
 'Erik von Detten',
 'Laurie Metcalf',
 'R. Lee Ermey',
 'Sarah Freeman',
 'Penn Jillette']

In [176]:
# Concatenar los df en base a la columna 'id'
directores = pd.merge(df[['id', 'title', 'release_date', 'budget', 'revenue', 'return']], df_directors, on='id')

directores.head()

,id,title,release_date,budget,revenue,return,director_name
0,862.0,Toy Story,1995-10-30,30000000.0,373554033.0,12.45,John Lasseter
1,8844.0,Jumanji,1995-12-15,65000000.0,262797249.0,4.04,Joe Johnston
2,15602.0,Grumpier Old Men,1995-12-22,0.0,0.0,0.00,Howard Deutch
3,31357.0,Waiting to Exhale,1995-12-22,16000000.0,81452156.0,5.09,Forest Whitaker
4,11862.0,Father of the Bride Part II,1995-02-10,0.0,76578911.0,0.00,Charles Shyer


Exportamos cada dataframe en archivos csv.

In [177]:
fecha_estreno.to_csv('fecha_estreno.csv', index=False)
score_votos.to_csv('score_votos.csv', index=False)
actores.to_csv('actores.csv', index=False)
directores.to_csv('directores.csv', index=False)